# Problem Set 2

## by Arianna Michelangelo, Tatiana Bakwenye, Andrew Bennett

In [127]:
# Import libraries
import pandas as pd
import yaml
import numpy as np
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import random
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
import numpy as np
from sklearn.base import clone
import doubleml as dml
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
import pickle
seed(1234)

# Own preprocessing


### Data loading and EDA

In [91]:
# Load datasets
rhc_df = pd.read_csv('rhc.csv')
with open('confounders.yml', 'r') as file:
    confounders = yaml.safe_load(file)


In [92]:
rhc_df.shape

(5735, 63)

In [93]:
rhc_df.describe()

,Unnamed: 0,sadmdte,dschdte,dthdte,lstctdte,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,edu,surv2md1,das2d3pc,t3d30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,wtkilo1,adld3p,urin1,ptid
count,5735.000000,5735.000000,5734.000000,3722.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,1439.000000,2707.000000,5735.000000
mean,2868.000000,11638.686312,11660.050401,11753.869156,11781.257890,0.176635,0.178030,0.098344,0.067306,0.189887,0.044464,0.069922,0.032258,0.229468,0.269050,0.115432,0.034874,61.375883,11.678461,0.592450,20.496888,23.612206,54.667655,21.004185,78.520052,15.645165,115.178901,28.092118,37.618357,222.273710,3.092743,31.865479,2.267067,2.133017,136.768963,4.066693,38.748975,7.388413,67.827817,1.182071,2192.453665,5134.006452
std,1655.696228,513.967751,513.447322,538.812330,524.094168,0.381393,0.382571,0.297805,0.250573,0.392246,0.206141,0.255037,0.176700,0.420527,0.443505,0.319570,0.183476,16.681929,3.145831,0.195354,5.317186,10.052266,19.956434,30.266794,38.046582,11.867515,41.244941,14.078041,1.774704,114.953087,0.783277,8.363786,4.801538,2.053080,7.655160,1.028353,13.183445,0.109812,29.055534,1.819057,1525.140006,2972.206379
min,1.000000,10754.000000,10757.000000,10757.000000,10756.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.041990,0.000000,0.000000,11.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.000000,11.599610,0.299988,2.000000,0.099991,0.099991,101.000000,1.099854,1.000000,6.579102,0.000000,0.000000,0.000000,5.000000
25%,1434.500000,11163.500000,11184.000000,11267.000000,11316.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.149490,10.000000,0.470947,16.058590,16.000000,41.000000,0.000000,50.000000,8.398438,97.000000,14.000000,36.093750,133.312500,2.599609,26.097656,0.799927,1.000000,132.000000,3.399902,31.000000,7.339844,56.299990,0.000000,1110.000000,2561.500000
50%,2868.000000,11759.000000,11777.000000,11831.500000,11868.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.047000,12.000000,0.628000,19.746090,30.000000,54.000000,0.000000,63.000000,14.099609,124.000000,30.000000,38.093750,202.500000,3.500000,30.000000,1.009766,1.500000,136.000000,3.799805,37.000000,7.399998,70.000000,0.000000,1927.000000,5131.000000
75%,4301.500000,12097.000000,12120.000000,12208.000000,12244.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,73.932465,13.000000,0.743000,23.429690,30.000000,67.000000,41.000000,115.000000,20.048828,141.000000,38.000000,39.000000,316.625000,3.500000,36.296875,1.399902,2.399902,142.000000,4.599609,42.000000,7.459961,83.699950,2.000000,2955.000000,7689.000000
max,5735.000000,12441.000000,12560.000000,12783.000000,12644.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,101.847960,30.000000,0.962000,33.000000,30.000000,147.000000,100.000000,259.000000,192.000000,250.000000,100.000000,43.000000,937.500000,29.000000,66.187500,58.195312,25.097656,178.000000,11.898438,156.000000,7.769531,244.000000,7.000000,9000.000000,10278.000000


In [94]:
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,sex,edu,surv2md1,das2d3pc,t3d30,dth30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,swang1,wtkilo1,dnr1,ninsclas,resp,card,neuro,gastr,renal,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,0,0,0,1,0,0,0,1,0,0,0,70.25098,Male,12.000000,0.640991,23.50000,30,No,46,0,41.0,22.097656,124,10.0,38.69531,68.00000,3.500000,58.000000,1.009766,1.199951,145,4.000000,40.0,7.359375,No RHC,64.69995,No,Medicare,Yes,Yes,No,No,No,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,1,0,0,0,0,0,0,0,1,1,0,78.17896,Female,12.000000,0.755000,14.75195,30,No,50,0,63.0,28.898438,137,38.0,38.89844,218.31250,2.599609,32.500000,0.699951,0.599976,137,3.299805,34.0,7.329102,RHC,45.69998,No,Private & Medicare,No,No,No,No,No,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,0,0,0,0,0,0,0,1,1,0,0,46.09198,Female,14.069916,0.317000,18.13672,30,No,82,0,57.0,0.049995,130,40.0,36.39844,275.50000,3.500000,21.097656,1.009766,2.599609,146,2.899902,16.0,7.359375,RHC,0.00000,No,Private,No,Yes,No,No,No,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,0,0,0,0,0,0,0,0,1,0,0,75.33197,Female,9.000000,0.440979,22.92969,30,No,48,0,55.0,23.296875,58,26.0,35.79688,156.65625,3.500000,26.296875,0.399963,1.699951,117,5.799805,30.0,7.459961,No RHC,54.59998,No,Private & Medicare,Yes,No,No,No,No,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,0,0,0,0,0,0,0,0,0,0,0,67.90997,Male,9.945259,0.437000,21.05078,2,Yes,72,41,65.0,29.699219,125,27.0,34.79688,478.00000,3.500000,24.000000,1.009766,3.599609,126,5.799805,17.0,7.229492,RHC,78.39996,Yes,Medicare,No,Yes,No,No,No,No,No,No,No,No,NaN,64.0,white,Under $11k,11


### Categorical treatment

In [95]:
rhc_df['ortho'] = rhc_df['ortho'].map({'No': 0, 'Yes': 1})
rhc_df['trauma'] = rhc_df['trauma'].map({'No': 0, 'Yes': 1})
rhc_df['seps'] = rhc_df['seps'].map({'No': 0, 'Yes': 1})
rhc_df['hema'] = rhc_df['hema'].map({'No': 0, 'Yes': 1})
rhc_df['meta'] = rhc_df['meta'].map({'No': 0, 'Yes': 1})
rhc_df['renal'] = rhc_df['renal'].map({'No': 0, 'Yes': 1})
rhc_df['gastr'] = rhc_df['gastr'].map({'No': 0, 'Yes': 1})
rhc_df['neuro'] = rhc_df['neuro'].map({'No': 0, 'Yes': 1})
rhc_df['card'] = rhc_df['card'].map({'No': 0, 'Yes': 1})
rhc_df['resp'] = rhc_df['resp'].map({'No': 0, 'Yes': 1})
rhc_df['dth30'] = rhc_df['dth30'].map({'No': 0, 'Yes': 1})
rhc_df['ca'] = rhc_df['ca'].map({'No': 0, 'Yes': 1})
rhc_df['sex'] = rhc_df['sex'].map({'Male': 0, 'Female': 1})
rhc_df['death'] = rhc_df['death'].map({'No': 0, 'Yes': 1})
rhc_df['swang1'] = rhc_df['swang1'].map({'No RHC': 0, 'RHC': 1})
rhc_df['dnr1'] = rhc_df['dnr1'].map({'No': 0, 'Yes': 1})


In [96]:
# Display unique values of categorical variables

# Define the function
def calculate_unique_counts(dataframe, columns):
    unique_counts = {}
    for column in columns:
        unique_values_count = dataframe[column].value_counts()
        unique_counts[column] = {'unique_count': dataframe[column].nunique(), 'value_counts': unique_values_count}
        # Print the results
        print(f"Number of unique values in {column}: {unique_counts[column]['unique_count']}")
        print(f"Value counts for {column}:")
        print(unique_values_count)
    return unique_counts

# List of columns for which we want to calculate unique counts
columns_to_check = ['cat1', 'cat2', 'race', 'income', 'ninsclas']

# Calculate unique counts for the specified columns
unique_counts = calculate_unique_counts(rhc_df, columns_to_check)

Number of unique values in cat1: 9
Value counts for cat1:
ARF                  2490
MOSF w/Sepsis        1227
COPD                  457
CHF                   456
Coma                  436
MOSF w/Malignancy     399
Cirrhosis             224
Lung Cancer            39
Colon Cancer            7
Name: cat1, dtype: int64
Number of unique values in cat2: 6
Value counts for cat2:
MOSF w/Sepsis        826
MOSF w/Malignancy    229
Coma                  90
Cirrhosis             38
Lung Cancer           15
Colon Cancer           2
Name: cat2, dtype: int64
Number of unique values in race: 3
Value counts for race:
white    4460
black     920
other     355
Name: race, dtype: int64
Number of unique values in income: 4
Value counts for income:
Under $11k    3226
$11-$25k      1165
$25-$50k       893
> $50k         451
Name: income, dtype: int64
Number of unique values in ninsclas: 6
Value counts for ninsclas:
Private                1698
Medicare               1458
Private & Medicare     1236
Medicaid  

In [97]:
# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to the specified column and update the original dataframe
rhc_df['income'] = le.fit_transform(rhc_df['income'])

In [98]:
# Initialize TargetEncoder with smoothing
encoder = ce.TargetEncoder(cols=['cat1', 'cat2'], smoothing=10)

# Fit and transform the encoder on the specified columns with the target variable
rhc_df[['cat1', 'cat2']] = encoder.fit_transform(rhc_df[['cat1', 'cat2']], rhc_df['death'])

In [99]:
# Create dummy variables for 'race' and 'ninsclas' columns
race_dummies = pd.get_dummies(rhc_df['race'], prefix='race')
ninsclas_dummies = pd.get_dummies(rhc_df['ninsclas'], prefix='ninsclas')

# Concatenate the dummy variables with the original dataframe
rhc_df = pd.concat([rhc_df, race_dummies, ninsclas_dummies], axis=1)

# Drop the original 'race' and 'ninsclas columns if needed
rhc_df.drop(['race', 'ninsclas'], axis=1, inplace=True)


In [100]:
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,sex,edu,surv2md1,das2d3pc,t3d30,dth30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,swang1,wtkilo1,dnr1,resp,card,neuro,gastr,renal,meta,hema,seps,trauma,ortho,adld3p,urin1,income,ptid,race_black,race_other,race_white,ninsclas_Medicaid,ninsclas_Medicare,ninsclas_Medicare & Medicaid,ninsclas_No insurance,ninsclas_Private,ninsclas_Private & Medicare
0,1,0.614880,0.616979,1.0,11142,11151.0,NaN,11382,0,0,0,0,0,1,0,0,0,1,0,0,0,70.25098,0,12.000000,0.640991,23.50000,30,0,46,0,41.0,22.097656,124,10.0,38.69531,68.00000,3.500000,58.000000,1.009766,1.199951,145,4.000000,40.0,7.359375,0,64.69995,0,1,1,0,0,0,0,0,0,0,0,0.0,NaN,3,5,0,0,1,0,1,0,0,0,0
1,2,0.641402,0.616979,0.0,11799,11844.0,11844.0,11844,1,1,1,0,0,0,0,0,0,0,1,1,0,78.17896,1,12.000000,0.755000,14.75195,30,0,50,0,63.0,28.898438,137,38.0,38.89844,218.31250,2.599609,32.500000,0.699951,0.599976,137,3.299805,34.0,7.329102,1,45.69998,0,0,0,0,0,0,0,0,1,0,0,NaN,1437.0,3,7,0,0,1,0,0,0,0,0,1
2,3,0.909774,0.721550,1.0,12083,12143.0,NaN,12400,0,0,0,0,0,0,0,0,0,1,1,0,0,46.09198,1,14.069916,0.317000,18.13672,30,0,82,0,57.0,0.049995,130,40.0,36.39844,275.50000,3.500000,21.097656,1.009766,2.599609,146,2.899902,16.0,7.359375,1,0.00000,0,0,1,0,0,0,0,0,0,0,0,NaN,599.0,1,9,0,0,1,0,0,0,0,1,0
3,4,0.590763,0.616979,0.0,11146,11183.0,11183.0,11182,1,0,0,0,0,0,0,0,0,0,1,0,0,75.33197,1,9.000000,0.440979,22.92969,30,0,48,0,55.0,23.296875,58,26.0,35.79688,156.65625,3.500000,26.296875,0.399963,1.699951,117,5.799805,30.0,7.459961,0,54.59998,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,0,0,1,0,0,0,0,0,1
4,5,0.641402,0.616979,0.0,12035,12037.0,12037.0,12036,1,0,0,0,0,0,0,0,0,0,0,0,0,67.90997,0,9.945259,0.437000,21.05078,2,1,72,41,65.0,29.699219,125,27.0,34.79688,478.00000,3.500000,24.000000,1.009766,3.599609,126,5.799805,17.0,7.229492,1,78.39996,1,0,1,0,0,0,0,0,0,0,0,NaN,64.0,3,11,0,0,1,0,1,0,0,0,0


In [101]:
rhc_df.shape

(5735, 70)

### Missing Data

In [102]:
# Identify features with missing values
features_with_missing_values = rhc_df.columns[rhc_df.isnull().any()].tolist()

# Display the count of missing values for each feature
missing_values_count = rhc_df[features_with_missing_values].isnull().sum()

print("Features with missing values:")

print(features_with_missing_values)

print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
['ca', 'dschdte', 'dthdte', 'adld3p', 'urin1']

Count of missing values for each feature:
ca          384
dschdte       1
dthdte     2013
adld3p     4296
urin1      3028
dtype: int64


In [103]:
# Drop columns since they have too many NaN
columns_to_exclude = ['cat2', 'adld3p', 'Unnamed: 0']
rhc_numerical = rhc_df.drop(columns=columns_to_exclude)

# Impute NaN with KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(rhc_numerical)
imputed_data = imputer.fit_transform(rhc_numerical)

imputed_df = pd.DataFrame(imputed_data, columns=rhc_numerical.columns)


In [104]:
# Check of it worked
features_with_missing_values = imputed_df.columns[imputed_df.isnull().any()].tolist()
missing_values_count = imputed_df[features_with_missing_values].isnull().sum()
print("Features with missing values:")
print(features_with_missing_values)
print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
[]

Count of missing values for each feature:
Series([], dtype: float64)


In [105]:
imputed_df.columns

Index(['cat1', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte', 'death',
       'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx', 'renalhx',
       'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx', 'amihx', 'age',
       'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30', 'aps1',
       'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1', 'alb1',
       'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1', 'swang1',
       'wtkilo1', 'dnr1', 'resp', 'card', 'neuro', 'gastr', 'renal', 'meta',
       'hema', 'seps', 'trauma', 'ortho', 'urin1', 'income', 'ptid',
       'race_black', 'race_other', 'race_white', 'ninsclas_Medicaid',
       'ninsclas_Medicare', 'ninsclas_Medicare & Medicaid',
       'ninsclas_No insurance', 'ninsclas_Private',
       'ninsclas_Private & Medicare'],
      dtype='object')

### Standardization

In [108]:
# Initialize StandardScaler and fit it to imputed_df
scaler = StandardScaler().fit(imputed_df)

# Transform imputed_df using the fitted scaler
imputed_df_scaled = scaler.transform(imputed_df)

In [113]:
imputed_df.columns

Index(['cat1', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte', 'death',
       'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx', 'renalhx',
       'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx', 'amihx', 'age',
       'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30', 'aps1',
       'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1', 'alb1',
       'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1', 'swang1',
       'wtkilo1', 'dnr1', 'resp', 'card', 'neuro', 'gastr', 'renal', 'meta',
       'hema', 'seps', 'trauma', 'ortho', 'urin1', 'income', 'ptid',
       'race_black', 'race_other', 'race_white', 'ninsclas_Medicaid',
       'ninsclas_Medicare', 'ninsclas_Medicare & Medicaid',
       'ninsclas_No insurance', 'ninsclas_Private',
       'ninsclas_Private & Medicare'],
      dtype='object')

### Prepare Data

In [115]:
# Remove 'ninsclas' and 'race' from the list of confounders
confounders = [col for col in confounders if col not in ['ninsclas', 'race']]

# Add the specified columns to the list of confounders
confounders.extend(['race_black', 'race_other', 'race_white',
                    'ninsclas_Medicaid', 'ninsclas_Medicare', 'ninsclas_Medicare & Medicaid',
                    'ninsclas_No insurance', 'ninsclas_Private', 'ninsclas_Private & Medicare'])

In [116]:
# Prepare the data
Y = imputed_df['death']
D = imputed_df['swang1']
X = imputed_df[confounders]

# Models

### Random Forest without GridSearch

In [122]:
# Define the cross-fitting function
def cross_fit_and_calculate_residuals(model, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred

    return residuals

# Models
model_y = RandomForestClassifier(random_state=42)
model_d = RandomForestClassifier(random_state=42)

# Calculate residuals
Y_residuals = cross_fit_and_calculate_residuals(model_y, X, Y)
D_residuals = cross_fit_and_calculate_residuals(model_d, X, D)


In [123]:
# Add a constant to the D residuals for the intercept in OLS
D_res_with_const = sm.add_constant(D_residuals)

# Linear regression
model = sm.OLS(Y_residuals, D_res_with_const)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     10.28
Date:                Wed, 21 Feb 2024   Prob (F-statistic):            0.00135
Time:                        14:01:48   Log-Likelihood:                -4287.2
No. Observations:                5735   AIC:                             8578.
Df Residuals:                    5733   BIC:                             8592.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1189      0.007    -17.218      0.0

In [124]:
# Prepare DoubleML data
dml_data = dml.DoubleMLData(imputed_df, y_col='death', d_cols='swang1', x_cols=confounders)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(model_d), ml_m=clone(model_y))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)


c:\Users\bakwe\my_env2\lib\site-packages\doubleml\double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: RandomForestClassifier(random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef  std err         t    P>|t|     2.5 %   97.5 %
swang1  0.052176  0.01591  3.279383  0.00104  0.020992  0.08336


c:\Users\bakwe\my_env2\lib\site-packages\doubleml\_utils_checks.py:204: UserWarning: Propensity predictions from learner RandomForestClassifier(random_state=42) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [125]:
# DoubleMLData and DoubleMLPLR initialization
confounders_ = [c for c in imputed_df.columns if c not in ["swang1", "death"]]
data_dml = dml.DoubleMLData(
    imputed_df, y_col="death", d_cols="swang1", x_cols=confounders_
)

ml_m = GradientBoostingClassifier()
ml_l = GradientBoostingRegressor()
dml_plr_obj = dml.DoubleMLPLR(data_dml, ml_l, ml_m)

# Hyperparameter setting
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [50, 100, 200]
max_depth = [1, 2, 3]

par_grids = {
    "ml_l": {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
    "ml_m": {
        "learning_rate": learning_rate,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
}

# Fine-tuning and effect estimation
dml_plr_obj.tune(param_grids=par_grids, tune_on_folds=True, search_mode="grid_search")
results = dml_plr_obj.fit()
results.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
swang1,0.022073,0.009498,2.323954,0.020128,0.003457,0.040689


### Random Forest with GridSearch

In [128]:
param_grid_classifier = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}


def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = RandomForestClassifier(random_state=42)
model_dc = RandomForestClassifier(random_state=42)

# Parameter grid for model optimization
param_grid_yc = param_grid_classifier # Define parameter grid for Y model
param_grid_dc = param_grid_classifier # Define parameter grid for D model

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, param_grid_yc)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, param_grid_dc)



In [129]:
# Running also regressors just to check if it runs better
param_grid_regressor = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Models 1
model_yr = RandomForestRegressor(random_state=42)
model_dr = RandomForestRegressor(random_state=42)

# Parameter grid for model optimization
param_grid_yr = param_grid_regressor # Define parameter grid for Y model
param_grid_dr = param_grid_regressor # Define parameter grid for D model 

Y_residualsr, optimized_model_yr = optimize_and_cross_fit(model_yr, X, Y, param_grid_yr)
D_residualsr, optimized_model_dr = optimize_and_cross_fit(model_dr, X, D, param_grid_dr)

c:\Users\bakwe\my_env2\lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\bakwe\my_env2\lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\bakwe\my_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\bakwe\my_env2\lib\site-packages\sklearn\metrics\_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "c:\Users\bakwe\my_env2\lib\site-packages\sklearn\utils\_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\bakwe\my_env2\lib\site-packages\sklearn\metrics\_classification.py", line 220, in accurac

In [130]:
filename = 'rfclassificatormodel_y.sav'
pickle.dump(optimized_model_yc, open(filename, 'wb'))
filename2 = 'rfclassificatormodel_d.sav'
pickle.dump(optimized_model_dc, open(filename2, 'wb'))
filename3 = 'rfregressormodel_y.sav'
pickle.dump(optimized_model_yr, open(filename3, 'wb'))
filename4 = 'rfregressormodel_d.sav'
pickle.dump(optimized_model_dr, open(filename4, 'wb'))


In [131]:
# Both classifiers

# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model = sm.OLS(Y_residualsc, D_res_with_constc)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     14.92
Date:                Wed, 21 Feb 2024   Prob (F-statistic):           0.000113
Time:                        16:01:56   Log-Likelihood:                -4193.1
No. Observations:                5735   AIC:                             8390.
Df Residuals:                    5733   BIC:                             8404.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1520      0.007    -22.379      0.0

In [132]:
# Prepare DoubleML data
dml_data = dml.DoubleMLData(imputed_df, y_col='death', d_cols='swang1', x_cols=confounders)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(optimized_model_dc), ml_m=clone(optimized_model_yc))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)


c:\Users\bakwe\my_env2\lib\site-packages\doubleml\double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=200,
                       random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef   std err         t     P>|t|     2.5 %    97.5 %
swang1  0.043448  0.015781  2.753207  0.005901  0.012518  0.074378


# With professor's preprocessing code

In [ ]:
rhc = pd.read_csv("rhc.csv")
with open("confounders.yml", "r") as f:
    confounders = yaml.safe_load(f)

rhc["swang1"] = (rhc["swang1"] == "RHC").astype(int)
rhc["death"] = (rhc["death"] == "Yes").astype(int)
rhc.groupby("swang1")["death"].mean()

rhc_numerical = pd.get_dummies(rhc[['swang1', 'death'] + confounders], dtype=float, drop_first=True)

In [ ]:
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]

# Prepare the data
Y = rhc_numerical['death']
D = rhc_numerical['swang1']
X = rhc_numerical[confounders_]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

par_grids = {
    'learning_rate' : [0.001, 0.01, 0.1], 
    'n_estimators' : [50, 100, 200], 
    'max_depth' : [1, 2, 3]
}

def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = GradientBoostingClassifier(random_state=42)
model_dc = GradientBoostingClassifier(random_state=42)

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, par_grids)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, par_grids)

In [ ]:
# both classificators

import statsmodels.api as sm

# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model = sm.OLS(Y_residualsc, D_res_with_constc)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.108
Date:                Tue, 20 Feb 2024   Prob (F-statistic):             0.0238
Time:                        11:54:57   Log-Likelihood:                -4309.1
No. Observations:                5735   AIC:                             8622.
Df Residuals:                    5733   BIC:                             8636.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1173      0.007    -17.209      0.0

In [ ]:
from sklearn.base import clone
import doubleml as dml

# Prepare DoubleML data
dml_data = dml.DoubleMLData(rhc_numerical, y_col='death', d_cols='swang1', x_cols=confounders_)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(optimized_model_dc), ml_m=clone(optimized_model_yc))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)

/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/doubleml/double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: GradientBoostingClassifier(n_estimators=200, random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef   std err         t     P>|t|    2.5 %    97.5 %
swang1  0.043382  0.016277  2.665246  0.007693  0.01148  0.075285


In [ ]:
# DoubleMLData and DoubleMLPLR initialization
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]
data_dml = dml.DoubleMLData(
    rhc_numerical, y_col="death", d_cols="swang1", x_cols=confounders_
)

ml_m = GradientBoostingClassifier()
ml_l = GradientBoostingRegressor()
dml_plr_obj = dml.DoubleMLPLR(data_dml, ml_l, ml_m)

# Hyperparameter setting
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [50, 100, 200]
max_depth = [1, 2, 3]

par_grids = {
    "ml_l": {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
    "ml_m": {
        "learning_rate": learning_rate,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
}

# Fine-tuning and effect estimation
dml_plr_obj.tune(param_grids=par_grids, tune_on_folds=True, search_mode="grid_search")
results = dml_plr_obj.fit()
results.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
swang1,0.042929,0.013305,3.226583,0.001253,0.016852,0.069006
